In [14]:
pip install dash


In [15]:
# Import required libraries
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

In [16]:
# Load airline data from a public URL into a pandas DataFrame
# Note: Replace this URL with a local file path if running offline
airline_data = pd.read_csv(
    'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv',
    encoding="ISO-8859-1",
    dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str}
)


In [17]:
# Initialize the Dash application
app = dash.Dash(__name__)

In [18]:
# Define the layout of the app
app.layout = html.Div(children=[
    html.H1('Flight Delay Time Statistics', style={'textAlign': 'center', 'color': '#503D36', 'font-size': 30}),

    # Year input field
    html.Div(["Input Year: ", dcc.Input(id='input-year', value='2010', type='number', style={'height':'35px', 'font-size': 20})],
             style={'font-size': 20, 'textAlign': 'center'}),
    html.Br(),

    # Charts organized into rows and columns for responsive layout
    html.Div([
        html.Div([dcc.Graph(id='carrier-plot')], style={'flex': 1, 'padding': '10px'}),
        html.Div([dcc.Graph(id='weather-plot')], style={'flex': 1, 'padding': '10px'})
    ], style={'display': 'flex', 'justify-content': 'center'}),

    html.Div([
        html.Div([dcc.Graph(id='nas-plot')], style={'flex': 1, 'padding': '10px'}),
        html.Div([dcc.Graph(id='security-plot')], style={'flex': 1, 'padding': '10px'})
    ], style={'display': 'flex', 'justify-content': 'center'}),

    html.Div([dcc.Graph(id='late-plot')], style={'width':'80%', 'padding': '10px', 'margin': '0 auto'})
])

In [19]:
# Helper function to calculate average delays
def compute_info(airline_data, entered_year):
    df = airline_data[airline_data['Year'] == int(entered_year)]
    avg_car = df.groupby(['Month', 'Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month', 'Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month', 'Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month', 'Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month', 'Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late

# Callback function to update graphs based on input year
@app.callback(
    [Output('carrier-plot', 'figure'),
     Output('weather-plot', 'figure'),
     Output('nas-plot', 'figure'),
     Output('security-plot', 'figure'),
     Output('late-plot', 'figure')],
    [Input('input-year', 'value')]
)
def update_graphs(entered_year):
    # Calculate the average delays for the selected year
    avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_info(airline_data, entered_year)

    # Create line plots for each type of delay
    carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average Carrier Delay (minutes) by Airline')
    weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average Weather Delay (minutes) by Airline')
    nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS Delay (minutes) by Airline')
    sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average Security Delay (minutes) by Airline')
    late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average Late Aircraft Delay (minutes) by Airline')

    # Return all figures as a list
    return [carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]

# Run the app on a local server
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>